In [ ]:
### ONE CHANNEL ###

from telethon import TelegramClient, events, errors
from pprint import pprint
import asyncio
import csv
import os
import pandas as pd
from telethon import TelegramClient


# Your API ID and Hash
api_id = 'XXX'  # Replace with your API ID
api_hash = 'XXXX'

#channel = '@rusich_army'
# Initialize the client
client = TelegramClient('session_name', api_id, api_hash)


### FORWARDED MESSAGES ONLY ###

# List to store message data
messages_info = []
messages_get_all = []



In [ ]:

# Function to process MessageReactions and format them nicely
def format_reactions(message_reactions):
    if message_reactions is not None and message_reactions.results:
        # Initialize an empty dictionary to store the reactions
        reactions_dict = {}
        # Loop through each reaction in the results
        for reaction in message_reactions.results:
            if reaction.reaction is not None:
                # Check the type of reaction and get the appropriate identifier
                if hasattr(reaction.reaction, 'emoticon'):
                    emoticon = reaction.reaction.emoticon
                elif hasattr(reaction.reaction, 'emoji_id'):
                    emoticon = f"custom_emoji_{reaction.reaction.emoji_id}"
                else:
                    emoticon = "unknown"
                
                count = reaction.count
                # Format each reaction as "emoticon:count"
                reactions_dict[emoticon] = count
            else:
                # text "No reactions" if there are no reactions
                return "No reactions"
        
        # Format the entire dictionary into a string with each item separated by ': '
        formatted_reactions = ': '.join(f"{key}{value}" for key, value in reactions_dict.items())
        return formatted_reactions
    return "No reactions"


In [ ]:

### FORWARDED MESSAGES ONLY ###

# List to store message data
messages_info = []
messages_get_all = []

async def fetch_messages(channel_name, csv_file=f'messages_{CHANNEL}.csv'):
    print("Fetching messages from channel:", channel_name)
    last_message_id = None

    # Check if the CSV file exists and load the last processed message ID
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        if not df.empty:
            last_message_id = df['Message_id'].max()

    async with client:
        # Start the client
        await client.start()
        # Get the channel
        channel = await client.get_entity(channel_name)

        # Fetch messages and check if they are forwarded
        if last_message_id:
            async for message in client.iter_messages(channel, min_id=last_message_id):
                await process_message(message, channel, channel_name, csv_file)
        else:
            async for message in client.iter_messages(channel):
                await process_message(message, channel, channel_name, csv_file)

        # Disconnect the client
        await client.disconnect()

async def process_message(message, channel, channel_name, csv_file):
    message = await client.get_messages(channel, ids=message.id)
    ##### FORWARDED MESSAGES ONLY #####
    if message.forward:
        # Get the sender's user or channel object
        sender = await message.get_sender()
        sender_name = sender.username if sender.username else (sender.title if hasattr(sender, 'title') else 'Unknown')

        # Try to get the forwarder's user or channel object
        forwarder_name = 'Unknown'
        if message.forward.original_fwd:
            try:
                if message.forward.from_id:
                    forwarder = await client.get_entity(message.forward.from_id)
                    forwarder_name = forwarder.username if hasattr(forwarder, 'username') and forwarder.username else (forwarder.title if hasattr(forwarder, 'title') else 'Unknown')
                else:
                    forwarder_name = 'Unknown'
            except (ValueError, errors.ChannelPrivateError):
                forwarder_name = 'Inaccessible, Private or Deleted Channel'

        my_dict = { 
            "Sender": sender_name, 
            "Forwarder": forwarder_name,
            "Date": message.date,
            "Message_id": message.id,
            "channel_id": channel.id, # same as peer_id, but just numbers
            "channel": channel_name, # dshrg2
            "channgel_title": channel.title, # ДШРГ Русич	
            "channel.has_geo": channel.has_geo,
            "Text": message.text,
            "mentioned": message.mentioned,
            "noForwards": message.noforwards,
            "views": message.views,
            "forwards": message.forwards,
            "reactions": format_reactions(message.reactions),
            "media_unread": message.media_unread,
        }
        print("Date:", my_dict['Date'])
        # Append the new message data to the CSV file immediately
        append_to_csv(csv_file, my_dict)

def append_to_csv(csv_file, data_dict):
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data_dict.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(data_dict)

# Use await directly to run the coroutine
await fetch_messages('@NeoficialniyBeZsonoV')